In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [40]:
data = pd.read_excel(".\LoadData.xlsx")
print(f'Shape : {data.shape}')
data.head()


Shape : (2184, 3)


,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [41]:
a=[]
b=[]
for i in range(1,data.shape[0]):
    a.append(data["Load (kW)"].iloc[i-1])
    b.append(data["Load (kW)"].iloc[i])

In [42]:
data = pd.DataFrame({'Previous Hour' : a, 'Present Hour' : b})
data.head()

,Previous Hour,Present Hour
0,5551.82208,4983.17184
1,4983.17184,4888.39680
2,4888.39680,5072.95872
3,5072.95872,5196.25980
4,5196.25980,5641.29720


In [43]:
max_x = np.max(data['Previous Hour'])
max_y = np.max(data['Present Hour'])

min_x = np.min(data['Previous Hour'])
min_y = np.min(data['Present Hour'])

data['Previous Hour'] = (data['Previous Hour'] - min_x) / (max_x - min_x)
data['Present Hour'] = (data['Present Hour'] - min_y) / (max_y - min_y)

data.head()


,Previous Hour,Present Hour
0,0.397877,0.293800
1,0.293800,0.276454
2,0.276454,0.310234
3,0.310234,0.332801
4,0.332801,0.414254


In [44]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(data["Previous Hour"], data["Present Hour"], test_size=0.1, random_state=5)

In [45]:
import math
m = np.random.uniform(-4, 4)
c = np.random.uniform(-4, 4)
n = 0.04
e = 1E-7
v = 0.7
epochs = 700
em2 = 0
ec2 = 0

for i in range(epochs):

    for j in range(X_Train.shape[0]):
        gm = -1 * (Y_Train.iloc[j] - m * X_Train.iloc[j] - c) * X_Train.iloc[j]

        # Calculating Grad C
        gc = -1 * (Y_Train.iloc[j] - m * X_Train.iloc[j] - c)

        # Calculating updated values of gm2 and gc2
        em2 = (v * em2) + (1-v) * (gm * gm)
        ec2 = (v * ec2) + (1-v) * (gc * gc)

        # Updating m and c values
        m -= (n * gm) / math.sqrt(e + em2)
        c -= (n * gc) / math.sqrt(e + ec2)


In [46]:
print(f'M value is {m} and C value is : {c}')

M value is 0.8925636442921766 and C value is : 0.004634821612165153


In [47]:
train_p = []
test_p = []
for i in range(X_Train.shape[0]):
    train_p.append(m * X_Train.iloc[i] + c)

for i in range(X_Test.shape[0]):
    test_p.append(m * X_Test.iloc[i] + c)

# Denormalization
train_p = [i * (max_y - min_y) + min_y for i in train_p]
test_p = [i * (max_y - min_y) + min_y for i in test_p]
Y_Train = [i * (max_y - min_y) + min_y for i in Y_Train]
Y_Test = [i * (max_y - min_y) + min_y for i in Y_Test]


In [48]:
# Error Calculation

train_MSE = 0
train_MAE = 0

for i in range(len(Y_Train)):
    train_MAE+=abs(train_p[i] - Y_Train[i])
for i in range(len(Y_Train)):
    train_MSE+=(train_p[i] - Y_Train[i]) ** 2

train_MAE/=len(Y_Train)
train_MSE/=len(Y_Train)
train_RMSE = train_MSE ** 0.5

print('Training Error :')
print(f'MAE : {train_MAE}')
print(f'MSE : {train_MSE}')
print(f'RMSE : {train_RMSE}')

test_MAE = 0
test_MSE = 0

for i in range(len(Y_Test)):
    test_MAE += abs(test_p[i] - Y_Test[i])
for i in range(len(Y_Test)):
    test_MSE += (test_p[i] - Y_Test[i]) ** 2

test_MAE /=len(Y_Test)
test_MSE /=len(Y_Test)
test_RMSE = test_MSE ** 0.5

print('Testing Error :')
print(f'MAE : {test_MAE}')
print(f'MSE : {test_MSE}')
print(f'RMSE : {test_RMSE}')

Training Error :
MAE : 492.3671618898641
MSE : 525328.2836526543
RMSE : 724.7953391493727
Testing Error :
MAE : 473.8868154390221
MSE : 498662.60393577267
RMSE : 706.1604661376709


In [49]:
load = float(input('Enter the load at previous hour : '))
load = (load - min_x) / (max_x - min_x)
prediction = m * load + c
prediction = (prediction * (max_y - min_y)) + min_y
print(f'Predicted load at present hour is {prediction} ')

Predicted load at present hour is 5322.32670337247 
